# 단순 선형 회귀 (학습 성능 개선)
- 텐서플로우의 학습성능을 개선하기 위해 아래의 기능을 사용할 수 있음
1. `데이터 표준화`: 학습률 향상에 도움을 줌
2. `콜백 함수`: 모델의 학습 방향, 저장 시점, 학습 정지 시점 등에 관한 상황을 모니터링하기 위한 도구

## 1. 데이터 정규화 (Normalization)
### 1) 데이터 정규화의 이해
- 데이터 정규화란? 
    - 모든 데이터가 동일한 정도의 스케일(중요도)로 반영되도록 해주는 처리
- 정규화를 해야하는 이유
    - 머신러닝 알고리즘은 데이터가 가진 feature(특성)들을 비교하여 데이터의 패턴을 찾는다
    - 이 때, 데이터가 가진 feature의 스케일이 심하게 차이가 나는 경우 수많은 학습단계를 거쳐 최적값에 도달하게 된다
    - 데이터에 `정규화 처리를 적용`하면 `쉽게 최적값에 도달`할 수 있으며 `학습률을 상대적으로 높여서 사용`할 수 있기 때문에 빠르게 훈련시킬 수 있다.

### 2) 데이터 정규화 방법
#### 1- 최소-최대 정규화 (Min-Max Normalization)
- 모든 feature에 대해 각각의 `최솟값 0`, `최대값 1`로, 그리고 `다른 값들은 0과 1 사이의 값`으로 변환
- 예) 어떤 특성의 `최솟값이 20`이고 `최대값이 40`인 경우, 30이라는 데이터가 있을 경우, 20은 `0`, 40은 `1`로 환산되기 때문에 30은 중간치인 `0.5`로 변환됨

## 2. 콜백함수
- 모델의 학습 방향, 저장 시점, 학습 정지 시점 등에 관한 상황을 모니터링하기 위한 도구
### 1) 콜백함수 사용 방법
- 콜백 정의

    ```python
    callbacks = [  콜백1, 콜백2, ... 콜백n  ]
    ```
- 학습을 위한 `fit() 함수`에서 `callbacks 파라미터`에 미리 정의한 리스트를 지정

    ```python
    callbacks = [콜백1, 콜백2, ..., 콜백n]
    model.fit(x_train, 
              y_train, 
              validation_data = (x_val, y_val), 
              epochs = 500, 
              callbacks=callbacks)```
- 혹은

    ```python
    model.fit(x_train,
              y_train,
              validation_data = (x_val, y_val),
              epochs = 500,
              callbacks = [콜백1, 콜백2, ..., 콜백n])```

### 2) 콜백함수 종류
#### 1- EarlyStopping()
- 모델 학습 시에 지정된 기간 동안 모니터링하는 평가지표에서 더 이상 성능 향상이 일어나지 않는 경우 학습을 스스로 중단

    ```python
    EarlyStopping(monitor='평가지표', patience=10, verbose=1)```

- patience: 지정한 수만큼의 기간에서 평가지표의 향상이 일어나지 않을 경우 학습을 중단 (기간=에폭)
    - 예) `patience=10`일 때, 10에폭 동안 성능 향상이 일어나지 않으면 학습을 중단. 즉, `10회 이상 성능향상이 발견되지 않으면 중단`
- verbose: 콜백의 수행 과정 노출 여부를 지정
    - 0: 아무런 표시 하지 않음(기본값)
    - 1: 프로그래스바로 표시
    - 2: 매 에폭마다 수행과정을 자세하게 출력함
#### 2- ReduceLROnPlateau()
- EarlyStopping 콜백과 같이 patience 인자를 지정하여, 지정된 기간 동안 평가지표에서 성능 향상이 일어나지 않으면 학습률을 조정하는 콜백

    ```python
    ReduceLROnPlateau(monitor='평가지표', factor=0.1,
                      patience=10, min_lr=0, verbose=1)```

- factor: 학습률 조정에 사용되는 값 (새로운 학습률 = factor  * 기존 학습률)
- patience: 지정한 수만큼의 기간에서 성능 향상이 일어나지 않을 경우, 학습률을 조정
- min_lr: 학습률의 하한 (lower limit)을 지정
- verbose: 콜백의 수행 과정 노출 여부를 지정
#### 3- ModelCheckpoint()
- 지정한 평가지표를 기준으로 `가장 뛰어난 성능을 보여주는 모델을 저장` 할 때 사용

    ```python
    ModelCheckpoint(filepath, monitor='평가지표', verbose=1,
                    save_best_only = True|False, save_weights_only = False)```
- filepath: 모델의 저장 경로를 지정
- save_best_only: True인 경우, 가장 성능이 뛰어난 모델만 저장. 그보다 좋지 않은 모델의 경우는 덮어쓰지 않음
- save_weights_only: 모델의 가중치만 저장
## 3. 단순선형회귀 수행
### 1) 패키지 준비하기

In [ ]:
import sys, os
sys.path.append('../../')

from pandas import read_excel, DataFrame
from matplotlib import pyplot as plt
import seaborn as sb
import numpy as np

# 데이터를 훈련용과 테스트용으로 나누는 기능
from sklearn.model_selection import train_test_split
# 데이터 정규화를 위한 패키지
from sklearn.preprocessing import MinMaxScaler

# 순서층을 구성하는 모델 객체 생성 기능
from tensorflow.keras.models import Sequential
# 모델 객체에 학습층을 쌓기 위한 클래스
from tensorflow.keras.layers import Dense
# 학습에 대한 콜백함수 참조
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import ModelCheckpoint

### 2) 데이터셋 준비

In [ ]:
origin = read_excel('https://data.hossam.kr/E04/cars.xlsx')
origin.head()

### 3) 데이터 전처리
- 결측치 확인

In [ ]:
origin.isna().sum()

> 결측치 없음

### 4) 탐색적 데이터 분석
- 기본통계 확인

In [ ]:
origin.describe()

- 산점도 그래프와 추세선 확인
    - `seaborn.regplot()`

In [ ]:
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 12
plt.rcParams['axes.unicode_minus'] = False

plt.figure(figsize=(10, 5))
sb.regplot(data=origin, x='speed', y='dist', color='orange')
plt.grid()

plt.show()
plt.close()

> 곡선보다는 선형 분포에 더 가까우므로 단순선형회귀모델을 적용하기로 함

### 5) 데이터셋 분할
- 랜덤시드 고정

In [1]:
np.random.seed(777)

NameError: name 'np' is not defined

- 훈련 데이터(독립변수)와 레이블(종속변수) 구분하기

In [ ]:
x = origin.drop(['dist'], axis = 1)
y = origin[['dist']]
print('훈련데이터 크기:', x.shape, '/ 레이블 크기:', y.shape)

- (추가) 데이터 정규화(표준화) 수행

In [ ]:
# 독립변수 표준화
x_scaler = MinMaxScaler()
x_scale = x_scaler.fit_transform(x)
x_scale

# 종속변수 표준화
y_scaler = MinMaxScaler()
y_scale = y_scaler.fit_transform(y)
y_scale

- 훈련 데이터와 검증 데이터로 분할

In [2]:
x_train, x_test, y_train, y_test = train_test_split(x_scale, 
                                                    y_scale, 
                                                    test_size = 0.3, 
                                                    random_state=777)
print('훈련용 데이터셋 크기: %d, 검증용 데이터셋 크기: %d' % (len(x_train), len(x_test)))

NameError: name 'ᄐ' is not defined

### 6) 모델 개발
- 모델 정의

In [ ]:
my_model = Sequential()
# 1차원의 데이터를 입력으로 받고, 32개의 출력을 가지는 첫 번때 Dense 층
my_model.add(Dense(32, activation = 'relu', input_shape = (1,)))
# 하나의 값을 출력
# -> 정답의 범위가 정해지지 않기 때문에 활성화 함수는 linear
# -> linear는 기본값이므로 생략 가능
my_model.add(Dense(1, activation = 'linear'))

my_model.compile(optimizer='adam', 
                 loss='mse',
                 metrics=['mae'])
my_model.summary()

In [3]:
os.getcwd()

NameError: name 'os' is not defined

In [ ]:
# result = my_model.fit(x_train, y_train, epochs=500, validation_data=(x_test,y_test))
# 파일이 저장될 경로(폴더) 지정 (한글, 공백, 점(.)이 포함되어 잇을 경우 에러 발생함)

checkpoint_path = os.path.join('D:\\tensorflow_checkpoint\\model07-cp-{epoch:04d}-ckpt')

result = my_model.fit(x_train, 
                      y_train, 
                      epochs=500, 
                      validation_data=(x_test,y_test),
                      callbacks=[EarlyStopping(mointor='val_loss',
                                               patience=10,
                                               verbose=1),
                                ReduceLROnPlateau(monitor='val_loss',
                                                  patience=3,
                                                  factor = 0.5,
                                                  min_lr=0.0001,
                                                  verbose=1),
                                ModelCheckpoint(filepath=checkpoint_path,
                                                monitor='val_loss',
                                                verbose=1,
                                                save_best_only=True)])
# 학습 결과
result_df = DataFrame(result.history)
result_df['epochs']=result_df.index+1
result_df.set_index('epochs', inplace=True)
result_df

### 7) 학습 결과 평가
- 학습 결과 시각화

In [5]:
# 그래프 기본설정
plt.rcParams['font.family']='AppleGothic'
plt.rcParams['font.size']=12
plt.rcParams['axes.unicode_minus']=False

# 그래프를 그리기 위한 객체 생성
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5), dpi=150)

# 1) 훈련 및 검증 손실 그리기
sb.lineplot(x=result_df.index, 
            y='loss',
            data=result_df, 
            color='blue',
            label='훈련 손실률',
            ax=ax1)
sb.lineplot(x=result_df.index, 
            y='val_loss',
            data=result_df, 
            color='orange',
            label='검증 손실률',
            ax=ax1)
ax1.set_title('훈련 및 검증 손실률')
ax1.set_xlabel('반복회차')
ax1.set_ylabel('손실률')
ax1.grid()
ax1.legend()

# 2) 훈련 및 검증 절대오차 그리기
sb.lineplot(x=result_df.index,
            y='mae',
            data=result_df,
            color='blue',
            label='훈련 절대오차',
            ax=ax2)
sb.lineplot(x=result_df.index,
            y='val_mae',
            data=result_df,
            color='orange',
            label='검증 절대오차',
            ax=ax2)
ax2.set_title('훈련 및 검증 절대오차')
ax2.set_xlabel('반복회차')
ax2.set_ylabel('정확도')
ax2.grid()
ax2.legend()

plt.show()
plt.close()

NameError: name 'plt' is not defined

> epochs를 500으로 지정했지만 `일찍 학습을 멈춘 것을 확인`할 수 있다
>
> 즉, 학습을 `조기 종료`함으로서 `학습 성능을 향상`시켰다고 할 수 있다

- 모델 성능 평가

In [ ]:
evaluate1 = my_model.evaluate(x_train, y_train)
print('최종 훈련 손실률: %f, 최종 훈련 절대오차: %f' % (evaluate1[0], evaluate1[1]))

evaluate2 = my_model.evaluate(x_test, y_test)
print('최종 검증 손실률: %f, 최종 검증 절대오차: %f' % (evaluate1[0], evaluate2[1]))

### 8) 학습 결과 적용
- 테스트 데이터에 대한 예측 결과 산정

In [ ]:
results = my_model.predict(x_test)
print(results)

- 결과 데이터셋 구성

In [ ]:
kdf = DataFrame({'검증데이터': x_test.flatten(),
                 '실제값': y_test.flatten(),
                 '예측값': results.flatten()})

kdf['예측오차'] = kdf['실제값'] - kdf['예측값']

kdf

- 실제 결과값과 머신러닝에 의한 예측값 비교

In [ ]:
helper.regplot(x_left=kdf['검증데이터'],
               y_left=kdf['실제값'],
               y_left_pred=kdf['예측값'])

- 임의의 값에 대한 머신러닝 예측 결과

In [ ]:
# 속도가 50일 때의 제동거리를 예측해보자
my_speed = 50

# 독립변수 표준화에 사용한 객체를 활용
my_speed_scale = x_scaler.transform([[my_speed]])
my_speed_scale

results = my_model.predict(my_speed_scale)
results

# 예측 결과는 표준화된 값으로 나오기 때문에 원래의 단위로 되돌리기 위해 역변환
y_scaler.inverse_transform(results)